In [32]:
%load_ext autoreload
%aimport -torch, pandas, hydra, numpy, pickle, os
# %aimport src, src.metrics
%autoreload 2
import os
from hydra import initialize, compose
import pathlib
import pandas as pd
import numpy as np
import pickle
from tqdm.auto import tqdm

# from datasets.jobdataset import generate_dataset, _hfd5_from_dataframe
from src.data_process.job_hdf5 import hdf5_from_dataframe, get_career
import src.data_process.neg_sample as ng_sample
from src.utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_PREDICTION_COL
# from implicit_eval import microsoft_eval,model_infer_df
# from implicit.als import AlternatingLeastSquares
# from implicit.bpr import BayesianPersonalizedRanking
# from implicit.lmf import LogisticMatrixFactorization
from src.implicit_build import bpr, bpr, lmf
from src.metrics import ranking
from src.metrics.evaluate_ignite import model_infer2

with initialize(version_base=None, config_path="../../src/conf"):
    cfg = compose(config_name="config", overrides=['path.root=../../data/jobs'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import wandb
wandb.login()

wandb: Currently logged in as: tiyuok2023. Use `wandb login --relogin` to force relogin


True

In [34]:
%aimport

Modules to reload:
all-except-skipped

Modules to skip:
torch, pandas, hydra, numpy, pickle, os


# Create the the data pipe

In [36]:
df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))

In [37]:
df_train_pos[DEFAULT_RATING_COL] = 1

In [4]:
# df_train_pos[DEFAULT_USER_COL] = df_train_pos[DEFAULT_USER_COL].astype(str)
# df_train_pos[DEFAULT_ITEM_COL] = df_train_pos[DEFAULT_ITEM_COL].astype(str)
df_train_pos[DEFAULT_USER_COL] = df_train_pos[DEFAULT_USER_COL].astype("category")
df_train_pos[DEFAULT_ITEM_COL] = df_train_pos[DEFAULT_ITEM_COL].astype("category")

In [11]:
# hdf5_from_dataframe(df_train_pos, pathlib.Path(cfg.path.root, cfg.file.hdf5))

## Read data & Train

In [17]:
jobsid, usersid, user_job_app = get_career(pathlib.Path(cfg.path.leave_one_cf, cfg.leave_one_data.hdf5))
model_path = "./models"

In [18]:
# user_job_app = job_user_app.T.tocsr()
# bpr(model_path, user_job_app)
# als(model_path, user_job_app)
# lmf(model_path, user_job_app)

In [19]:
def read_train_gd_csv(data_testgd_path, usecols):
    test_gddf = pd.read_csv(data_testgd_path, usecols=usecols)
#     test_gddf[DEFAULT_USER_COL] = test_gddf[DEFAULT_USER_COL].astype('str')
#     test_gddf[DEFAULT_ITEM_COL] = test_gddf[DEFAULT_ITEM_COL].astype('str')
    return test_gddf

In [20]:
df_test_ori = pd.read_feather(pathlib.Path(cfg.path.leave_one_cf, cfg.leave_one_data.test_pos_neg))
# df_train =read_train_gd_csv('../../data/jobs/leave_one_train_neg.csv', usecols=[DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL])
# df_test =read_train_gd_csv('../../data/jobs/leave_one_test.csv', usecols=[DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL])

In [21]:
with open('./models/model_als.sav','rb') as pickle_in:
    model_als = pickle.load(pickle_in)
with open('./models/model_bpr.sav','rb') as pickle_in:
    model_bpr = pickle.load(pickle_in)
with open('./models/model_lmf.sav','rb') as pickle_in:
    model_lmf = pickle.load(pickle_in)

In [22]:
def cal_metrics(apps_true, jobsid, usersid,model, u_i_matrix, n, metric:dict):
    gt_item, reco_ind, pd_scores = model_infer2(df_true=apps_true, jobsid=jobsid, usersid=usersid, 
                                 model=model, u_i_matrix=user_job_app, n=cfg.params.neg_test+1) 
    precision = metric['precision'].compute(gt_pos=gt_item, pd_rank=reco_ind)
    recall = metric['recall'].compute(gt_pos=gt_item, pd_rank=reco_ind)
    ndcg = metric['ndcg'].compute(gt_pos=gt_item, pd_rank=reco_ind)
    auc_k = metric['auc_k'].compute(gt_pos=gt_item, pd_scores=pd_scores, pd_rank=reco_ind)
    auc = metric['auc'].compute(gt_pos=gt_item, pd_scores=pd_scores)
    hit = metric['hit'].compute(gt_pos=gt_item, pd_rank=reco_ind)
    map_k = metric['map_k'].compute(gt_pos=gt_item, pd_scores=pd_scores, pd_rank=reco_ind)
    
    return [precision, recall, ndcg, auc_k, auc, hit, map_k]

In [23]:
def get_results(metric_value, wandb_enable: False, model_name:str, project_name:str)->dict:
    precision_top = np.mean(np.array(list(metric_value.flat))[:,0])
    recall_top = np.mean(np.array(list(metric_value.flat))[:,1])
    ndcg_top = np.mean(np.array(list(metric_value.flat))[:,2])
    auc_top_k = np.mean(np.array(list(metric_value.flat))[:,3])
    auc = np.mean(np.array(list(metric_value.flat))[:,4])
    hit = np.mean(np.array(list(metric_value.flat))[:,5])
    map_k = np.mean(np.array(list(metric_value.flat))[:,6])
    print(f'Precision: {precision_top:.4f} \nRecall: {recall_top:.4f} \nNDCG: {ndcg_top:.4f} \nAUC_K: {auc_top_k:.4f} \
    \nAUC: {auc:.4f} \nHitRate: {hit:.4f} \nMAP_K: {map_k:.4f}')
    result_dict = {'Precision': precision_top, 'Recall': recall_top, 'NDCG': ndcg_top, 'AUC_K':auc_top_k, 'AUC':auc,\
                   'HitRate':hit, 'MAP_K':map_k}
    if wandb:
        wandb_log(model_name, project_name, result_dict)

In [30]:
def wandb_log(model_name:str, project_name:str, result:dict):
    wandb.init(project=project_name,
          name = model_name,
           tags= ['jrs','cf'],
           config = dict(cfg.params)
          )
    wandb.log(result)

In [25]:
apps_true = df_test_ori[df_test_ori['userid'].isin([1472090])]


In [27]:
tqdm.pandas()
k=cfg.params.topk
metric={'precision':ranking.Precision(k=k),'recall':ranking.Recall(k=k), 'ndcg':ranking.NDCG(k=k), \
        'auc_k':ranking.AUC_K(k=k),'auc':ranking.AUC(), 'hit':ranking.HitRate(k=k), 'map_k':ranking.MAP_K(k=k)}

In [28]:
metric_result = apps_true.groupby('userid').progress_apply(cal_metrics, jobsid=jobsid, usersid=usersid, 
                            model=model_bpr, u_i_matrix=user_job_app, 
                           n=cfg.params.neg_test+1, metric=metric)
metric_value_b = metric_result.values

  0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
restult_dict = get_results(metric_value_b, wandb_enable=True,model_name='bpr', project_name=cfg.name.cf_name)

Precision: 0.1000 
Recall: 1.0000 
NDCG: 0.3010 
AUC_K: 0.1111     
AUC: 0.9048 
HitRate: 1.0000 
MAP_K: 0.1111


In [29]:
metric_result = df_test_ori.groupby('userid').progress_apply(cal_metrics, jobsid=jobsid, usersid=usersid, 
                            model=model_als, u_i_matrix=user_job_app, 
                           n=cfg.params.neg_test+1, metric=metric)
metric_value_a = metric_result.values

100%|██████████| 74091/74091 [09:27<00:00, 130.64it/s]


In [30]:
restult_dict = get_results(metric_value_b, wandb_enable=True,model_name='als', project_name=cfg.name.cf_name)

Precision: 0.0848 
Recall: 0.8477 
NDCG: 0.7233 
AUC_K: 0.7690     
AUC: 0.9213 
HitRate: 0.8477


In [31]:
metric_result = df_test_ori.groupby('userid').progress_apply(cal_metrics, jobsid=jobsid, usersid=usersid, 
                            model=model_lmf, u_i_matrix=user_job_app, 
                           n=cfg.params.neg_test+1, metric=metric)
metric_value_l = metric_result.values

100%|██████████| 74091/74091 [09:00<00:00, 137.11it/s]


In [32]:
restult_dict = get_results(metric_value_b, wandb_enable=True,model_name='lmf', project_name=cfg.name.cf_name)

Precision: 0.0786 
Recall: 0.7861 
NDCG: 0.6728 
AUC_K: 0.7111     
AUC: 0.8925 
HitRate: 0.7861


In [ ]:
Precision: 0.0786 
Recall: 0.7861 
NDCG: 0.6728 
AUC_K: 0.7111     
AUC: 0.8925 
HitRate: 0.1283